
# DA320 Assignment 3: SQL Server
Jon Kaimmer  
DA320  
Winter2022


 ### Summary
 My program scrapes web data for movies released in 2010. This data is stored in an array of movie Objects and then each movie Object is asked to update itself to a local SQL Server. 
Process:
 - Data Fields (variable and object declarations)
 - Webscraping (Data is scraped from the web and housed in a local variable string of text)
 - Regular expressions are used to search for and extract desired movie data 
 - Each movie's data is aggregated and assembled locally in a python class object 
 - Finally, each Python object calls a self defined "uploadToSQL" function that handels the data transfer to our SQL Server database. 

### DATA FIELDS

In [ ]:
#IMPORTS
import urllib3
import re
import json
import pyodbc

webLinkDict = {
    2010 : "https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2010&sort=desc&view=detailed"
}
numberOfMovies = 100
movieArray = []
SQLquery="INSERT INTO movies (movieName,description,releaseDate,metascore,movieImage) values (?,?,?,?,?)"

#MOVIE OBJECT DEFINITION 
#Individual objects store the data that is scraped from the metacritic website for each movie. This data is warehoused in the array named "movieArray"
class movie():
    def __init__(self, title, movieNumber, releaseDate, metacriticScore, image, movieSummary):
        self.title = title
        self.movieNumber = movieNumber
        self.releaseDate = releaseDate
        self.metacriticScore = metacriticScore
        self.image= image
        self.movieSummary = movieSummary
    def toString(self):
        print(f"{self.title}  Movie#{self.movieNumber}")
        print(f"{self.title}  Released: {self.releaseDate}")
        print(f"{self.title}  Metacritic Score: {self.metacriticScore}")
        print(f"{self.title}  Image: {self.image}")
        print(f"{self.title}  Summary: \n\t{self.movieSummary}")
        print("---")
    def parameterize(self):
        return (self.title, self.movieSummary, self.releaseDate, self.metacriticScore, self.image)
    def uploadToSQLServer(self, cursor):
        cursor.execute(SQLquery,self.parameterize())

#REGULAR EXPRESSIONS
#compile regular expressions into pattern objects
movieNumberREGEX = re.compile(r'\<span class="title numbered">*\s+(\d*)')  
movieTitleREGEX = re.compile(r'class="title"><h3>(.*?)<\/h3>')     
movieDateOfReleaseREGEX = re.compile(r'\s*<span>(\S+ \d+, \d+)')   
movieDescriptionREGEX = re.compile(r'<div class="summary">\n\s+(.*)')
movieMetacriticScoreREGEX = re.compile("<span class=\"title\">Metascore:<\\/span>\\s+<a class=\"metascore_anchor\" href=\"\\/movie\\/.*?\\/critic-reviews\">\s+<div class=\"metascore_w large movie positive\">(.*?)<\\/div>")
movieImageREGEX = re.compile(r'<a href=\"\/.*?<img src=\"(https:\/\/.*?)\" alt=\"')  

  
### PULL DOWN METACRITIC WEBPAGE

In [ ]:
# Construct a python request pool
http = urllib3.PoolManager()

# Initiate a web request
r = http.request('GET', webLinkDict[2010], headers={'User-Agent': 'Mozilla/5.0'})

# Convert the result from raw bytes to text
datastring = str(r.data, "utf-8")

### EXTRACT DATA USING REGULAR EXPRESSIONS

In [ ]:
list_movieTitles = re.findall(movieTitleREGEX, datastring)   #Extract list of movie titles
list_movieNumbers = re.findall(movieNumberREGEX, datastring)   #Extract list of movie numbers
list_releaseDates = re.findall(movieDateOfReleaseREGEX, datastring)   #Extract list of release dates
list_movieDescriptions = movieDescriptionREGEX.findall(datastring)   #Extract list of movie summaries
list_movieImages = movieImageREGEX.findall(datastring)   #Extract list of movie images
list_movieMetaCriticScores = movieMetacriticScoreREGEX.findall(datastring)   #Extract list of metacritic scores

### ASSEMBLE DATA LOCALLY IN PYTHON CLASS OBJECT

In [ ]:

#Load data into array of movieObjects.
#This happens in 4 steps:
    #Step 1 - iterate over the total number of movie objects
    #Step 2 - create a new instance of a movie class object
    #Step 3 - instantiate that new movie instance with the aggregated data from the lists of webscraped data. 
    #Step 4 - Newly created and filled movie object is appended to the array of movie objects

#movie(title, movieNumber, releaseDate, metacriticScore, image, movieSummary):
for i in range(len(list_movieNumbers)):
    movieArray.append( movie(
        list_movieTitles[i], 
        list_movieNumbers[i], 
        list_releaseDates[i],
        list_movieMetaCriticScores[i],
        list_movieImages[i],
        list_movieDescriptions[i]
        ))

### ESTABLISH CONNECTION TO SQL SERVER

In [ ]:
credentialLocation = "C:\\Users\\jonat\\OneDrive\\Documents\\GitHub\\WebScraping_JupyterNB\\credentials.json"
with open(credentialLocation, 'r') as myFile:
    data = myFile.read() #read file
    credentialDict = json.loads(data) #parse json file into a python dictionary
    userName = credentialDict['userName']
    password = credentialDict['password']

# Create the pyodbc bridge to our SQL database
#   Create a connection string to my local SQL Server
connectionString = (
            "Driver={SQL Server};"
            "Server=LAPTOP-V05JMA9V;"
            "Database=MoviesDB;"
            "Trusted_Connection=yes;"
            "UID=" + userName + ";"
            "PWD=" + password + ";"
            )
connection = pyodbc.connect(connectionString)

# Create cursor object
cursor = connection.cursor()

### INSERT DATA INTO SQL SERVER DATABASE

In [ ]:
#Data is currently stored in our array of movie objects. We need iterate over and send each ones data in a cursor.execute statement to our local SQL Server
#Breakdown:
    #Step 1 - iterate over the total number of movie objects
    #Step 2 - call self defined "uploadToSQLServer()" function. (Parameters: connection cursor) (Return value: N/A)
        #Step 2.1 - self.parameterize() formats and returns our SQL arguments (Parameters: N/A) (Return Value: TUPLE(self.title, self.movieSummary, self.releaseDate, self.metacriticScore, self.image))
        #Step 2.2 - self.uploadToSQLServer() calls the connection.execute command on our SQL statement and SQL arguments
    #End of Loop
    #Step 3 - commit changes to SQL server


for movie in movieArray:
    movie.uploadToSQLServer(cursor)
cursor.commit()

### PRINT OUTPUT

In [ ]:
#Loop over the movie array and use embedded "movieToString()" function to print our result. 
for movie in movieArray:
    movie.toString()